# ALL installs 

In [1]:
# ! pip install requests"
# ! pip install kafka-python
# ! pip install pyspark
# !pip  install -r requirements.txt

In [2]:
!pip freeze > requirements.txt

# ALL imports

In [3]:
from allsecrets import *
from titles import *
from coordinates import *
import json,requests
from json import dumps,loads 
from kafka import KafkaProducer, KafkaConsumer
import kafkaconnect

In [4]:
from pyspark.sql import SparkSession , functions as F

from pyspark.sql.functions import udf,col,countDistinct,date_format,row_number

from pyspark.sql.types import FloatType , StringType , IntegerType , StructType , StructField , TimestampType

from pyspark.sql.window import Window 

spark = SparkSession.builder.appName('Weather')\
    .getOrCreate()



your 131072x1 screen size is bogus. expect trouble


22/11/16 16:54:08 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.19.40.176 instead (on interface eth0)
22/11/16 16:54:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/16 16:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Json print function

In [5]:
def json_print(json_object):
    text = json.dumps(json_object, sort_keys=True, indent=4) # sort_keys=True means sort the keys in alphabetical order
    # indent 4 means 4 spaces for each indentation
    print(text)

## Openweather 5 days 3 hour forcast

In [7]:

parameters = {'lat': '27.7167', "lon": '85.3667', "appid": open_weather_api}
sample_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)

# save the response as json file
with open('sample_response.json', 'w') as f:
    json.dump(sample_response.json(), f)

    


weather_rdd= spark.sparkContext.parallelize([json.dumps(sample_response.json()) ]) # convert to rdd

#convert to spark dataframe
weather_df = spark.read.json(weather_rdd) # convert to spark dataframe

schema_weather = weather_df.schema # get the schema of the dataframe

#  save  schema_weather  to .json file
with open('schema_weather.json', 'w') as f:
    json.dump(schema_weather.jsonValue(), f, indent=4)

#  load  schema_weather  from .json file
with open('schema_weather.json', 'r') as f:
    schema_weather1 = F.StructType.fromJson(json.load(f)) 

# schema_weather1

# weather_df.printSchema()


In [8]:
#api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

for lat ,lon in lat_lon.items():
    parameters = {'lat': lat, "lon": lon, "appid": open_weather_api}
    weather_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)
    print (weather_response.url)
    print(weather_response.status_code)
    # json_print(weather_response.json())



http://api.openweathermap.org/data/2.5/forecast?lat=27.7167&lon=85.3667&appid=8cf7d4ce0b67a86cf3b125bc5c552a6d
200


# Test produces and consumer

## Test producer

In [27]:
# producer = KafkaProducer(bootstrap_servers=[
#                          'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'), key_serializer=lambda x: dumps(x).encode('utf-8'))
#                          # both key(header) and value are converted to json format

producer = KafkaProducer(bootstrap_servers=[
                         'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))
                         # both key(header) and value are converted to json format

for lat, lon in lat_lon.items():
    parameters = {'lat': lat, "lon": lon, "appid": open_weather_api}
    weather_response = requests.get(
        "http://api.openweathermap.org/data/2.5/forecast", params=parameters)

    producer.send('weather', value=weather_response.json())
    producer.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer

## Test consumer

In [37]:
# create consumer
# Consumer = KafkaConsumer('weather', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest',
#                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8'), key_deserializer=lambda x: loads(x.decode('utf-8'))))


Consumer = KafkaConsumer('weather', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))

In [38]:
# consume the data from the kafka broker and save to json file
for message in Consumer:
    message = message.value
    with open('weather.json', 'a') as f:
        json.dump(message, f)
        f.write(' ') # add space between each json object


KeyboardInterrupt: 

In [66]:
# close consumer to free up resources after reading the messages
Consumer.close()

In [25]:
Consumer.topics()

{'_confluent-command',
 '_confluent-telemetry-metrics',
 '_confluent_balancer_api_state',
 '_confluent_balancer_broker_samples',
 '_confluent_balancer_partition_samples',
 '_schemas',
 'connect-configs',
 'connect-offsets',
 'connect-status',
 'weather'}

In [25]:
# create topic
! kafka-topics --bootstrap-server localhost:9092 --create --topic weather --replication-factor 1 --partitions 1



Created topic weather.


In [26]:
# delete topic
! kafka-topics --bootstrap-server localhost:9092 --delete --topic weather

# Spark magic

## Unpacking and Exploding the  columns

In [187]:

weather_df = spark.read.json('weather.json')

nested_json_type=type (weather_df.select("city").take(1)[0][0]) # get the type of the nested json

# function to expand the nested json object only
def expand_json(df):
    for c in df.columns:
        if type(df.select(c).take(1)[0][0]) == nested_json_type:
            df = df.select("*", F.col(c+".*")).drop(c)
            print(c)
    return df
    
# weather_df.printSchema()

In [188]:

#explode "list" column  as it is in list  of dictionary conataing 40 records
weather_df = weather_df.select ("*", F.explode("list").alias("every_3_hr_weather")).drop("list")

weather_df.show(5)

+--------------------+---+---+-------+--------------------+
|                city|cnt|cod|message|  every_3_hr_weather|
+--------------------+---+---+-------+--------------------+
|{{27.7167, 85.366...| 40|200|      0|{{20}, 1668600000...|
|{{27.7167, 85.366...| 40|200|      0|{{24}, 1668610800...|
|{{27.7167, 85.366...| 40|200|      0|{{26}, 1668621600...|
|{{27.7167, 85.366...| 40|200|      0|{{2}, 1668632400,...|
|{{27.7167, 85.366...| 40|200|      0|{{2}, 1668643200,...|
+--------------------+---+---+-------+--------------------+
only showing top 5 rows



In [189]:
# LEVEL 1 Unpacking (city and every_3_hr_weather  column)
weather_df=expand_json(weather_df)
weather_df.show(5)

city
every_3_hr_weather
+---+---+-------+------------------+-------+-------+--------------+----------+----------+----------+--------+------+----------+-------------------+--------------------+----+------+---+----------+--------------------+-----------------+
|cnt|cod|message|             coord|country|     id|          name|population|   sunrise|    sunset|timezone|clouds|        dt|             dt_txt|                main| pop|  rain|sys|visibility|             weather|             wind|
+---+---+-------+------------------+-------+-------+--------------+----------+----------+----------+--------+------+----------+-------------------+--------------------+----+------+---+----------+--------------------+-----------------+
| 40|200|      0|{27.7167, 85.3667}|     NP|1283647|Baudhatinchule|         0|1668559213|1668597990|   20700|  {20}|1668600000|2022-11-16 12:00:00|{290.89, 867, 68,...|0.29|{0.23}|{n}|     10000|[{light rain, 10n...|{268, 1.74, 1.58}|
| 40|200|      0|{27.7167, 85.3667}|

In [190]:
#explode weather columnn ,it is in list  of dictionary conataing 1 record only
weather_df = weather_df.select ("*", F.explode("weather").alias("weather_dict")).drop("weather")

# LEVEL 2 Unpacking  (coord,clouds,main,rain,sys,wind,weather_dict column)
weather_df=expand_json(weather_df)


# rename column all to coludiness
weather_df = weather_df.withColumnRenamed("all", "cloudiness")
weather_df.show(5)

# weather_df.printSchema()

coord
clouds
main
rain
sys
wind
weather_dict
+---+---+-------+-------+-------+--------------+----------+----------+----------+--------+----------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+--------+---------+------+-------+--------+--------+----+---+---+----+-----+----------------+----+---+------+
|cnt|cod|message|country|     id|          name|population|   sunrise|    sunset|timezone|        dt|             dt_txt| pop|visibility|    lat|    lon|cloudiness|feels_like|grnd_level|humidity|pressure|sea_level|  temp|temp_kf|temp_max|temp_min|  3h|pod|deg|gust|speed|     description|icon| id|  main|
+---+---+-------+-------+-------+--------------+----------+----------+----------+--------+----------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+--------+---------+------+-------+--------+--------+----+---+---+----+-----+----------------+----+---+------+
| 40|200|      0|     NP|1283647|Baudhat

In [191]:
#  delete the column which is not required
weather_df = weather_df.drop("cod","message","cnt","pressure","3h","icon","timezone","population","country","id")

weather_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- sunrise: long (nullable = true)
 |-- sunset: long (nullable = true)
 |-- dt: long (nullable = true)
 |-- dt_txt: string (nullable = true)
 |-- pop: double (nullable = true)
 |-- visibility: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- cloudiness: long (nullable = true)
 |-- feels_like: double (nullable = true)
 |-- grnd_level: long (nullable = true)
 |-- humidity: long (nullable = true)
 |-- sea_level: long (nullable = true)
 |-- temp: double (nullable = true)
 |-- temp_kf: double (nullable = true)
 |-- temp_max: double (nullable = true)
 |-- temp_min: double (nullable = true)
 |-- pod: string (nullable = true)
 |-- deg: long (nullable = true)
 |-- gust: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- description: string (nullable = true)
 |-- main: string (nullable = true)



## Important functions

In [192]:
# function to convert the unix timestamp to date and time 
def convert_unix_to_date_time(df):
    for c in df.columns:
        if c == "dt":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
        elif c == "sunrise":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
        elif c == "sunset":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
    return df

# convert the unix timestamp to date and time
weather_df = convert_unix_to_date_time(weather_df)

weather_df.show(5)

weather_df.printSchema()

+--------------+-------------------+-------------------+-------------------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+---------+------+-------+--------+--------+---+---+----+-----+----------------+------+
|          name|            sunrise|             sunset|                 dt|             dt_txt| pop|visibility|    lat|    lon|cloudiness|feels_like|grnd_level|humidity|sea_level|  temp|temp_kf|temp_max|temp_min|pod|deg|gust|speed|     description|  main|
+--------------+-------------------+-------------------+-------------------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+---------+------+-------+--------+--------+---+---+----+-----+----------------+------+
|Baudhatinchule|2022-11-16 06:25:13|2022-11-16 17:11:30|2022-11-16 17:45:00|2022-11-16 12:00:00|0.29|     10000|27.7167|85.3667|        20|    290.89|       867|      68|     1016|291.25|   4.61|  291.25|  286.64|  n|268|1.74| 1.

## Question 1

In [194]:
Producer_save_table = KafkaProducer(bootstrap_servers=[
                         'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))

for row in weather_df.rdd.collect():
    Producer_save_table.send('cleaned_table', value=row.asDict())
    Producer_save_table.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer

In [ ]:

# Caused by: org.apache.kafka.connect.errors.DataException: JsonConverter with schemas.enable requires "schema" and "payload" fields and may not contain additional fields. If you are trying to deserialize plain JSON data, set schemas.enable=false in your converter configuration.

{
    "name": "cleaned_table-sink-connector",
    "config": {
        "connector.class": "io.confluent.connect.jdbc.JdbcSinkConnector",
        "connection.url": "jdbc:postgresql://localhost:5432/sink",
        "connection.user": "postgres",
        "connection.password":"1234",
        "auto.create": "true",
        "auto.evolve": "true",
        "table.name.format": "cleaned",
        "tasks.max": "1",
        "topics": "cleaned_table",
        "insert.mode": "upsert",
        "schema.ignore": "true"
        # <!-- #set schemas.enable=false in your converter configuration. -->
        "key.converter.schemas.enable": "false",

        

    }

}




lkl

